# [github.com/KentoNishi/stat364-postraining-compression](https://github.com/KentoNishi/stat364-postraining-compression)

By: Kento Nishi

---

I ran the ALPS method ([Alternating Lagrangian Pruning for Sparsity](https://github.com/mazumder-lab/ALPS)) that Professor Mazumder covered in his slide deck; he did not get to it during class time, due to time constraints. I thought it would be useful for me to play around with it for everyone else in the class to also learn from. The code below downloads the provided data, loads the model, runs ALPS pruning, and saves the outputs.

The layer-wise reconstruction objective the paper defines is as follows (pretrained weights $W_0$; calibration inputs/outputs $X,Y$):

$
\min_W\;\frac12\lVert XW^\top - Y\rVert_F^2\; +\;\frac\lambda2\lVert W - W_0\rVert_F^2
\quad\text{s.t.}\quad \lVert W\rVert_0 \le k
$

ALPS solves this with the ADMM optimizer, and applies it layer-by-layer to reach the target sparsity.


## Setup


In [1]:
# Paths and parameters
from pathlib import Path

MODEL_NAME = "facebook/opt-125m"
SPARSITY = 0.8
NSAMPLES = 8

MODEL_PATH = Path("./hf_cache")
DATA_PATH = Path("./data")
RESULTS_PATH = Path("./results")
PRUNED_PATH = Path("./pruned_models")

RESULTS_PATH.mkdir(parents=True, exist_ok=True)
PRUNED_PATH.mkdir(parents=True, exist_ok=True)
MODEL_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH.mkdir(parents=True, exist_ok=True)


## Data

In [2]:
# Download the authors' calibration data
import urllib.request
import zipfile

zip_url = "https://www.dropbox.com/scl/fi/6xg1voa7go9x2uds1y2mq/scd_data.zip?rlkey=8bwzshamiyvcvpd146ymkp0vc&dl=1"
zip_path = DATA_PATH / "scd_data.zip"

if not zip_path.exists():
    urllib.request.urlretrieve(zip_url, zip_path)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(DATA_PATH)

nested = DATA_PATH / "data"
if nested.exists():
    DATA_PATH = nested


## Run ALPS

In [3]:
# Allow model download (README: set cached=False)
from pathlib import Path

opt_path = Path("ALPS/opt.py")
text = opt_path.read_text()
text = text.replace("cached = True", "cached = False")
opt_path.write_text(text)


12731

In [4]:
# Run ALPS using the authors' script
import os
import subprocess
import sys

env = os.environ.copy()
env["HF_HUB_OFFLINE"] = "0"
env["TRANSFORMERS_OFFLINE"] = "0"
env["HF_DATASETS_OFFLINE"] = "0"
env["ALPS_SKIP_ZEROSHOT"] = "1"

cmd = [
    sys.executable,
    "ALPS/opt.py",
    MODEL_NAME,
    "c4",
    "ALPS",
    str(SPARSITY),
    "--model_path",
    str(MODEL_PATH),
    "--data_path",
    str(DATA_PATH),
    "--nsamples",
    str(NSAMPLES),
]

subprocess.check_call(cmd, env=env)


/home/kento/miniconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


cuda:0
loading data
facebook/opt-125m
Starting ...
Ready.
----
0 self_attn.k_proj


0 self_attn.v_proj
0 self_attn.q_proj


0 self_attn.out_proj
0 fc1


0 fc2


...

11 self_attn.q_proj
11 self_attn.out_proj


11 fc1


11 fc2


Evaluating ...
0
1
2
3
4
5
6
7
8
9
10
11
591.803955078125
Evaluating ...
0
1
2
3
4
5
6
7
8
9
10
11
772.3150634765625
loading data
facebook/opt-125m
Evaluating ...
0
1
2
3
4
5
6
7
8
9
10
11
250.8695068359375


0